# Building Your Predictor

The next step after preparing and importing your data via `Getting_Data_Ready.ipynb` is to build your first model.

The overall process for this is:

* Setup
* Create a Predictor
* Deploy a Predictor
* Obtain a Forecast

To get started, simply execute the cells below:


## Setup

Amazon Forecast is still in preview, to update to the latest functionality execute the cells below.

In [1]:
# Configures your AWS CLI for Amazon Forecast
!aws configure add-model --service-model file://../sdk/forecastquery-2019-08-12.normal.json --service-name forecastquery
!aws configure add-model --service-model file://../sdk/forecast-2019-08-12.normal.json --service-name forecast

Next import the standard Python Libraries that are used in this lesson.

In [2]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time

The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [3]:
session = boto3.Session(region_name='us-west-2') 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Create a Predictor

Now in the previous notebook, your data was imported to be used by Forecast, here we will once again define your dataset information and then start building your model or predictor.

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [4]:
project = 'electric_power_forecastdemo' # This should be the same as your previous notebook

In [5]:
predictorName= project+'_arima_algo'

In [6]:
forecastHorizon = 24

In [7]:
algorithmArn = 'arn:aws:forecast:::algorithm/ARIMA'

In [8]:
datasetGroupArn = "arn:aws:forecast:us-west-2:059124553121:dataset-group/util_power_forecastdemo_dsg" # Fill in the quotes from the output of the previous notebook.

In [9]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [10]:
predictorArn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [11]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


### Get Error Metrics

In [12]:
forecast.get_accuracy_metrics(PredictorArn=predictorArn)

{'PredictorEvaluationResults': [{'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 15.858717295125148,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.05510762517210228},
       {'Quantile': 0.5, 'LossValue': 0.17311171552235247},
       {'Quantile': 0.1, 'LossValue': 0.09249091407423402}]}},
    {'TestWindowStart': datetime.datetime(2014, 10, 30, 0, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2014, 10, 31, 0, 0, tzinfo=tzlocal()),
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 15.858717295125148,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.05510762517210228},
       {'Quantile': 0.5, 'LossValue': 0.17311171552235247},
       {'Quantile': 0.1, 'LossValue': 0.09249091407423402}]}}]}],
 'ResponseMetadata': {'RequestId': '9d079de8-c5a3-4900-bd1e-e9e1eadf26f6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 21 Aug 2019 13:53:

## Create a Forecast

Now create a forecast using the model that was trained

In [13]:
forecastName= project+'_arima_algo_forecast'

In [14]:
create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [15]:
while True:
    forecastStatus = forecast.describe_forecast(ForecastArn=forecastArn)['Status']
    print(forecastStatus)
    if forecastStatus != 'ACTIVE' and forecastStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


### Get Forecast

Once created, the forecast results are ready and you view them. 

In [16]:
print(forecastArn)
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecastArn,
    Filters={"item_id":"client_12"}
)
print(forecastResponse)

arn:aws:forecast:us-west-2:059124553121:forecast/electric_power_forecastdemo_arima_algo_forecast
{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2014-10-31T00:00:00', 'Value': 29.775606155395508}, {'Timestamp': '2014-10-31T01:00:00', 'Value': 24.531373977661133}, {'Timestamp': '2014-10-31T02:00:00', 'Value': 23.613906860351562}, {'Timestamp': '2014-10-31T03:00:00', 'Value': 18.986337661743164}, {'Timestamp': '2014-10-31T04:00:00', 'Value': 16.847822189331055}, {'Timestamp': '2014-10-31T05:00:00', 'Value': 12.550665855407715}, {'Timestamp': '2014-10-31T06:00:00', 'Value': 6.858987331390381}, {'Timestamp': '2014-10-31T07:00:00', 'Value': 13.320586204528809}, {'Timestamp': '2014-10-31T08:00:00', 'Value': 40.33671188354492}, {'Timestamp': '2014-10-31T09:00:00', 'Value': 79.19049835205078}, {'Timestamp': '2014-10-31T10:00:00', 'Value': 108.01700592041016}, {'Timestamp': '2014-10-31T11:00:00', 'Value': 111.27761840820312}, {'Timestamp': '2014-10-31T12:00:00', 'Value': 95.26617431640625}